In [ ]:
import json
from pathlib import Path
import xarray as xr
import pandas as pd

# -----------------------------
# CONFIG
# -----------------------------
NETCDF_FILE = Path("era5_italy_monthly_t2m_2021_2025.nc")
OUT_JSON = Path("italy_monthly_avg_temperature_c_2021_2025.json")

ROUND_DECIMALS = 1

# -----------------------------
# LOAD NETCDF
# -----------------------------
ds = xr.open_dataset(NETCDF_FILE)

if "t2m" not in ds.variables:
    raise ValueError(f"'t2m' not found. Available variables: {list(ds.variables)}")

t2m = ds["t2m"]

# -----------------------------
# MEAN OVER ALL NON-TIME DIMS
# -----------------------------
time_dim_candidates = [d for d in t2m.dims if "time" in d.lower()]
if not time_dim_candidates:
    raise ValueError(f"No time-like dimension found in: {t2m.dims}")

time_dim = time_dim_candidates[0]  # usually valid_time
dims_to_mean = [d for d in t2m.dims if d != time_dim]

t2m_mean_k = t2m.mean(dim=dims_to_mean)
t2m_mean_c = t2m_mean_k - 273.15

# -----------------------------
# TO DATAFRAME
# -----------------------------
df = t2m_mean_c.to_dataframe(name="avg_temp_c").reset_index()

# Time column
if "valid_time" in df.columns:
    time_col = "valid_time"
elif "time" in df.columns:
    time_col = "time"
else:
    raise ValueError(f"No time column found. Columns: {df.columns}")

df[time_col] = pd.to_datetime(df[time_col])
df["year"] = df[time_col].dt.year
df["month"] = df[time_col].dt.month

df = df[["year", "month", "avg_temp_c"]].sort_values(["year", "month"])

# -----------------------------
# BUILD CLEAN JSON RECORDS
# -----------------------------
records = [
    {
        "year": int(y),
        "month": int(m),
        "avg_temp_c": round(float(t), ROUND_DECIMALS)  # ✅ guaranteed clean
    }
    for y, m, t in df.itertuples(index=False, name=None)
]

# -----------------------------
# EXPORT JSON
# -----------------------------
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(records, f, indent=2)

print("Saved JSON:", OUT_JSON)
print("Preview:", records[:5])


✅ Saved JSON: italy_monthly_avg_temperature_c_2021_2025.json
✅ Preview: [{'year': 2021, 'month': 1, 'avg_temp_c': 11.7}, {'year': 2021, 'month': 2, 'avg_temp_c': 12.3}, {'year': 2021, 'month': 3, 'avg_temp_c': 14.3}, {'year': 2021, 'month': 4, 'avg_temp_c': 18.3}, {'year': 2021, 'month': 5, 'avg_temp_c': 23.4}]
